# PDF Crawler

In [41]:
%load_ext autoreload
%autoreload 2

import re
import os
import json
import time
import datetime
import numpy as np
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from tqdm import tqdm
from tqdm.notebook import tqdm, trange
import requests
from pathlib import Path
from PyPDF2 import PdfReader


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read submission list
Here we scrape the _notes_ , (list of all submissions) using OpenReview's API, way faster than Selenium-based scraping.


In [42]:
DATA_PATH = '../data/'
venue = 'ICLR.cc/2022/Conference'
venue_short = 'iclr2022'

# Read hdf5 file
df = pd.read_hdf(DATA_PATH + 'iclr2022_data_full_20221101.h5', key='df')

In [54]:

# Download pdfs from df['content.pdf']
def download_pdf(url, filename):
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)


def get_pdf_data(url, save_dir='temp/')
    # Make temp directory if not exis
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    BASE_URL = 'https://openreview.net/pdf?id='
    filename = save_dir / (id + '.pdf')
    download_pdf(BASE_URL + id, filename)

    reader = PdfReader(filename)
    number_of_pages = len(reader.pages)
    file_size = os.stat(filename)
    text_all = ''
    num_characters = 0
    for page in reader.pages:
        text = page.extract_text()
        # text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
        text_all += text
        num_characters += len(text)
    return {'text': text_all, 'num_characters': num_characters, 'num_pages': number_of_pages, 'file_size': file_size}

SyntaxError: expected ':' (4255663261.py, line 7)

In [ ]:
id = df['id'][1]

In [6]:


print("Size of file :", file_size.st_size, "bytes")
# conver total bytes to MB
print("Size of file :", file_size.st_size / (1024 * 1024), "MB")


In [11]:
page.keys()

dict_keys(['/Annots', '/Contents', '/MediaBox', '/Parent', '/Resources', '/Type'])